In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from fractions import Fraction
from natsort import natsorted
from ftfy import fix_text
import time
today = time.strftime("%Y_%m_%d")
from enrich_dimensions.rounds import rounds, rounding_inch_feet,rounding_lbs,rounding_w,rounding_oz, rounding_lb,rounding_gal, re_extract, curate, round_string_float, clean_data,reg_ex,rounding_period_after_unit    
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

customer_id = '5'
formatted_attribute = 'Height'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
curation_col = f'Q:{attribute}'

customer_name='%bedbathandbeyond%'
buckets = """Office Chairs"""
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name}
print('starting')
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
print(len(dfs))
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['LONG_DESCRIPTION']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
reg_ex(df)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']
df.drop_duplicates(subset='external_id', inplace=True)

import time
today = time.strftime("%Y_%m_%d")
print(today)

starting
39175
39175
2023_04_25


In [2]:
# x=df[(df['units'].astype(str)=='[]')&(df['size'].astype(str)=='[]')]
# print(len(x))
# x[500:1000]

In [3]:
# pat='''(?i)(\bled\b)|(ornament)|()'''
# df['size']=df['product_name'].apply(lambda x: re_extract(pat, x))
# size=df[df['size'].astype(str)!='[]']
# print(len(size))
# size

In [4]:
pat_na=r'''(?i)(\d\\?'(?!\]))|()'''
df['ft_long'] = df['long_desc'].apply(lambda x: re_extract(pat_na, x))
df['ft_name'] = df['product_name'].apply(lambda x: re_extract(pat_na, x))

# hmatches (1)

In [5]:
heights=df[df['height'].astype(str)!='[]']
heights['height']=heights['height'].apply(lambda x: natsorted(x)).apply(lambda x: re.sub(r'(?i)\\"\,\sHeight',' in',str(x)))
print(len(heights))
clean_data(heights,'height')

regex_pattern_blank=r'(?i)(\(?(?<![A-z])H\\?\s?x.{0,2}(?<![a-w])[A-z]\\?\s?x.{0,2}(?<![a-w])[A-z]\\?\s?\)?)|(\(?(?<![A-z])[A-z]\\?\s?x.{0,2}(?<![a-w])H\\?\s?x.{0,2}(?<![a-w])[A-z]\\?\s?\)?)|(\(?(?<![A-z])[A-z]\\?\s?x.{0,2}(?<![a-w])[A-z]\\?\s?x.{0,2}(?<![a-w])H\\?\s?\)?)'
df['units'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))

heights['height_finalz']=heights['two'].apply(lambda x: re.sub(r'(?i)(\\{1,2}"\s)','',str(x))).apply(lambda x: re.sub(r'''(\,\s?')|(\,(?!")(?!'))''',',"',str(x))).apply(lambda x: re.sub(r'\.\sin',' in',str(x))).apply(lambda x: re.sub(r'(\.\s)','',str(x)))
heights['height_finals']=heights['height_finalz']
heights['height_finals']=heights['height_finals'].apply(lambda x: re.sub(r'''(\,\s?')|(\,(?!")(?!'))''',',"',str(x)))
curate(heights, 'height_finals', formatted_attribute)

print(len(heights))
rounding_inch_feet(heights, curation_col,'a-eghj-mo-su-z')
heights_final=heights[heights['rounding'].astype(str)=='[]']
print(len(heights_final))
hmatches=heights_final[['external_id',curation_col]]
print(len(hmatches))

1795
Repeated External IDs: []
1795
1712
1712


In [6]:
# df[df['units'].astype(str)!='[]']['units'].explode().value_counts()
# print(len(unit))

# NA Values

In [7]:
regex_pattern_na=r'''(?i)((?<!Cord Measures.{10})\d+\.\d+[^A-z]{0,2}['"][^A-z]{0,2}[LWH]?)|((?<!Cord Measures.{10})\d+.?\d+\/\d+[^A-z]{0,2}['"][^A-z]{0,2}[LWH]?)|((?<!Cord Measures.{10})(?<!\.)(?<!\/)\d+(?!\.\d)[^A-z]{0,2}['"][^A-z]{0,2}[LWH]?)|((?<!Cord Measures.{10})(?<!\.)(?<!\/)\d+(?!\.)[^A-z]{0,2}['"][^A-z]{0,2}[LWH]?)|()'''
df['na_matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_na, x))

df_na=df[(df['na_matches'].astype(str)=='[]')&(df['blank_matches'].astype(str)=='[]')&(df['height'].astype(str)=='[]')&(df['tripple_long'].astype(str)=='[]')&(df['tripple_name'].astype(str)=='[]')&(df['double_name'].astype(str)=='[]')&(df['double_long'].astype(str)=='[]')&(df['blank_matches_long'].astype(str)=='[]')&(df['units'].astype(str)=='[]')]
print(len(df_na))
df_na[curation_col]='n/a'
matchesna=df_na[['external_id',curation_col]]

2629


# Trips

In [8]:
pat=r'(?i)(W x D x H)|(L x W x H)|(L\,\s?W\,\s?H)|((?<![A-z])[A-z]\s?x\s?[A-z]\s?x\s?H)|()'
df['LWH'] = df['long_desc'].apply(lambda x: re_extract(pat, x))

pat=r'(?i)((?=x)\s?.{0,9}(?=\(W\s?x\s?D\s?x\s?H\)))|((?=x)\s?.{0,9}(?=\(L\s?x\s?W\s?x\s?H\)))|()'
df['trips'] = df['long_desc'].apply(lambda x: re_extract(pat, x))

pat=r'''(?i)((?<!\d)(?<!\.)\d+'\s?\d+")|()'''
df['foot_inch'] = df['long_desc'].apply(lambda x: re_extract(pat, x))

In [9]:
x=df[df['LWH'].astype(str)=='[]']
print(len(x))
blank=x[x['blank_matches_long'].astype(str)!='[]']
print(len(blank))
blank[0:500]

34686
29837


buckets external_id  \
1                              Shelf Dividers/Organizers   201187101   
2                              Shelf Dividers/Organizers   200925973   
3                              Shelf Dividers/Organizers   200925981   
4                              Shelf Dividers/Organizers   200925970   
6                                                 Fryers   201179006   
11                                                Fryers   201179021   
16                            Reusable Food Storage Bags   200900889   
17                            Reusable Food Storage Bags   200900890   
18                            Reusable Food Storage Bags   200900891   
19                            Reusable Food Storage Bags   200900892   
20                            Reusable Food Storage Bags   200900893   
21                            Reusable Food Storage Bags   200900894   
22                            Reusable Food Storage Bags   201039274   
23                            Reusable Food Storage Bags   201039275   
24                            Reusable Food Storage Bags   201039276   
25                            Reusable Food Storage Bags   201039277   
26                            Reusable Food Storage Bags   201039278   
27                            Reusable Food Storage Bags   201039279   
28                            Reusable Food Storage Bags   201039280   
29                            Reusable Food Storage Bags   201039281   
30                            Reusable Food Storage Bags   201039282   
31                            Reusable Food Storage Bags   201039283   
32                            Reusable Food Storage Bags   201039284   
34                     Flush & Semi-Flush Mount Lighting   201152586   
37                 Play Pens, Safety Gates, & Enclosures   200881891   
39                 Play Pens, Safety Gates, & Enclosures   200883246   
42                 Play Pens, Safety Gates, & Enclosures   200901672   
43                 Play Pens, Safety Gates, & Enclosures   200901794   
44                 Play Pens, Safety Gates, & Enclosures   200901795   
45                 Play Pens, Safety Gates, & Enclosures   200901797   
48                 Play Pens, Safety Gates, & Enclosures   200861252   
49                 Play Pens, Safety Gates, & Enclosures   200861247   
54                 Play Pens, Safety Gates, & Enclosures   200751831   
63                 Play Pens, Safety Gates, & Enclosures   200791021   
64                 Play Pens, Safety Gates, & Enclosures   200791011   
65                 Play Pens, Safety Gates, & Enclosures   200793361   
66                 Play Pens, Safety Gates, & Enclosures   200791013   
84                 Play Pens, Safety Gates, & Enclosures   200791014   
90                 Play Pens, Safety Gates, & Enclosures   200629321   
100                Play Pens, Safety Gates, & Enclosures   200721210   
101                Play Pens, Safety Gates, & Enclosures   200721209   
107                Play Pens, Safety Gates, & Enclosures   200721137   
108                Play Pens, Safety Gates, & Enclosures   200722447   
109                Play Pens, Safety Gates, & Enclosures   201044180   
110                Play Pens, Safety Gates, & Enclosures   201044181   
150                Play Pens, Safety Gates, & Enclosures   201178922   
172                Play Pens, Safety Gates, & Enclosures   201126805   
179                Play Pens, Safety Gates, & Enclosures   200720636   
198                Play Pens, Safety Gates, & Enclosures   200945998   
199                Play Pens, Safety Gates, & Enclosures   200946000   
201                Play Pens, Safety Gates, & Enclosures   200946005   
203                Play Pens, Safety Gates, & Enclosures   200540717   
205  Cookware & Bakeware Replacement Parts & Accessories   200627587   
206  Cookware & Bakeware Replacement Parts & Accessories   200627588   
207  Cookware & Bakeware Replacement Parts & Accessories   201024682   
208  Cookware & B

In [10]:
tripple_long=df[df['tripple_long'].astype(str)!='[]']#.explode().value_counts()
tripple_long[0:500]
print(len(tripple_long))
trip=tripple_long[tripple_long['LWH'].astype(str)=='[]']
print(len(trip))

6294
1920


In [11]:
# df[df['foot_inch'].astype(str)!='[]']

In [12]:
foot_inch=df[(df['foot_inch'].astype(str)!='[]')]
foot_inch['height'].apply(lambda x: re.sub(r'','',str(x)))
foot_inch

Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, LONG_DESCRIPTION, length, height, width, diameter, blank_matches, blank_matches_long, tripple_name, tripple_long, double_name, double_long, mm_name, mm_long, units, ft_long, ft_name, na_matches, LWH, trips, foot_inch]
Index: []

In [13]:
pat=r'(?i)(W x D x H)|(L x W x H)|(L\,\s?W\,\s?H)|()'
df['LWH'] = df['long_desc'].apply(lambda x: re_extract(pat, x))

df_wdh=df[(df['na_matches'].astype(str)!='[]')&(df['height'].astype(str)=='[]')&(df['LWH'].astype(str)!='[]')&(df['foot_inch'].astype(str)=='[]')]
print(len(df_wdh))
df_name=df_wdh[['external_id','product_name','long_desc','height','blank_matches','blank_matches_long','tripple_name','tripple_long','double_name','na_matches','LWH','trips']]
df_name['Height']=df_name['trips'].apply(lambda x: re.sub(r'''\[(?:'|")?.*x\s?''','["',str(x))).apply(lambda x: re.sub(r'''(?i)(\\)''','',str(x))).apply(lambda x: re.sub(r'''(?<=\[)\s?'|(?<=\,)\s?'|'\s?(?=\])|'\s?(?=\,)|(?:''|“)|("")''','"',str(x)))
df_name['Height']=df_name['Height'].apply(lambda x: re.sub(r'''(?i)((?<=\d)'\s?(?=\d"))''',"'",str(x)))


731


In [14]:
foot_inch=df[(df['foot_inch'].astype(str)!='[]')]
foot_inch['height'].apply(lambda x: re.sub(r'','',str(x)))
foot_inch

Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, LONG_DESCRIPTION, length, height, width, diameter, blank_matches, blank_matches_long, tripple_name, tripple_long, double_name, double_long, mm_name, mm_long, units, ft_long, ft_name, na_matches, LWH, trips, foot_inch]
Index: []

In [15]:
# df_name

In [16]:
# # df_name[curation_col]=(df_name.Height.str.split("'").str[0].astype(int) * 12) + (df_name.Height.str.split("'").str[1].astype(int))

# print(len(df_name))
# curate(df_name,curation_col,'height')
# match_trips=df_name[['external_id',curation_col]]
# match_trips

In [17]:
# def frac2string(s):
#     i, f = s.groups(0)
#     f = round_string_float(Fraction(f))
#     return str(int(i) + round_string_float(float(f)))
# # measure='This is a fraction: 100 1/64'
# # [re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)]

In [18]:
# def remove_exponent(num):
#     return num.to_integral() if num == num.to_integral() else num.normalize()

# def round_string_float(x):
#     try:
#         return remove_exponent(Decimal(x).quantize(TWOPLACES))
#     except:
#         return x  

# def inch_feet(s):
#     i, f = s.groups(0)
#     i = int(i)*12
#     return str(int(i) + round_string_float(float(f)))

In [19]:
# # .apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
# x=df_name['Height'][0:1]
# x=["4"]
# # [re.sub(r'''(?:(?<!\d)(?<!\.)\d+'\s?\d+")''', inch_feet, x)]
# i,j=str(x).split(r"'")
# i

In [20]:
# df_name['Height'][0:1].apply(lambda x: [re.sub(r'''(?:((?<!\d)(?<!\.)\d+'\s?\d+")''', inch_feet, x)])

In [21]:
# df_name=df[['external_id','product_name','long_desc','height','blank_matches','blank_matches_long','tripple_name','tripple_long','double_name','na_matches']]
# regex_pattern_blank=r'(?i)(\d+(?:.Inch)\s?x\s?\d+(?:.Inch))|()'
# df_name['new_blank_matches'] = df_name['blank_matches'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
# df_names=df_name[(df_name['new_blank_matches'].astype(str)!='[]')]
# print(len(df_names))

# regex_pattern_blank=r"(?i)(\d+'\s?x\s?\d+')|()"
# df_name['new_blank_matches_feet'] = df_name['blank_matches'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
# df_names_ft=df_name[(df_name['new_blank_matches_feet'].astype(str)!='[]')]
# print(len(df_names_ft))

In [22]:
# from fractions import Fraction
# def frac2string(s):
#     i, f = s.groups(0)
#     f = round_string_float(Fraction(f))
#     return str(int(i) + round_string_float(float(f)))

In [23]:
# df_names_ft['blanks']=df_names_ft['new_blank_matches_feet'].apply(lambda x: re.sub(r'''(?i)((\-inc?h?)|(inch)|(\s?\\{0,3}\s?"\s?)|((?<!\d)\s(?!\d))|([a-wyz])|(”)|(\[)|(\])|(\\{0,2}xa?0(?!\.)))|(\))|(\()''','',str(x))).apply(lambda x: f'"[{x}]"').astype(str)
# df1 = df_names_ft['blanks'].str[2:-2].str.split(',').explode().str.split(r'(?i)x', expand=True).fillna('')
# df1[1]=df1[1].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, str(x))]).apply(lambda x: re.sub(r'''('?\[(?:"|')\[(?:"|'))''','["',str(x))).apply(lambda x: re.sub(r'''('\](?:"|')\])|(?:\[|\]|"|'|\\+)''','',str(x)))
# df1[0]=df1[0].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, str(x))]).apply(lambda x: re.sub(r'''('?\[(?:"|')\[(?:"|'))''','["',str(x))).apply(lambda x: re.sub(r'''('\](?:"|')\])|(?:\[|\]|"|'|\\+)''','',str(x)))

# x=df1[1].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)]).to_list()
# y=df1[0].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)]).to_list()

# df_names_ft[['Max', 'Min']] = pd.concat([df1.astype(float).max(1).groupby(level=0).apply(list), 
#                                 df1.astype(float).min(1).groupby(level=0).apply(list)], axis=1)


# clean_data(df_names_ft,'Max')
# df_names_ft['doubs']=df_names_ft['two'].apply(lambda x: re.sub(r'\[\s?\"\s?\[\s?(:"|")?','',str(x))).apply(lambda x: re.sub(r'"?\]"\]','"]',str(x))).apply(lambda x: re.sub(r'((?<=\d)")|(in)',' ft',str(x)))
# curate(df_names_ft,'doubs','height')
# df_names_ft[curation_col]=df_names_ft[curation_col].apply(lambda x: re.sub(r'(in)',' ft',str(x)))
# rounding_inch_feet(df_names_ft,curation_col,'a-eghj-mo-su-z')
# print(len(df_names_ft))
# # df_names_ft[['blank_matches','Q:height']]
# match_names_ft=df_names_ft[['external_id',curation_col]]

In [24]:
# df_names['blanks']=df_names['new_blank_matches'].apply(lambda x: re.sub(r'''(?i)((\-inc?h?)|(inch)|(\s?\\{0,3}\s?"\s?)|((?<!\d)\s(?!\d))|([a-wyz])|(”)|(\[)|(\])|(\\{0,2}xa?0(?!\.)))|(\))|(\()''','',str(x))).apply(lambda x: f'"[{x}]"').astype(str)
# df1 = df_names['blanks'].str[2:-2].str.split(',').explode().str.split(r'(?i)x', expand=True).fillna('')
# df1[1]=df1[1].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, str(x))]).apply(lambda x: re.sub(r'''('?\[(?:"|')\[(?:"|'))''','["',str(x))).apply(lambda x: re.sub(r'''('\](?:"|')\])|(?:\[|\]|"|'|\\+)''','',str(x)))
# df1[0]=df1[0].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, str(x))]).apply(lambda x: re.sub(r'''('?\[(?:"|')\[(?:"|'))''','["',str(x))).apply(lambda x: re.sub(r'''('\](?:"|')\])|(?:\[|\]|"|'|\\+)''','',str(x)))

# x=df1[1].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)]).to_list()
# y=df1[0].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)]).to_list()

# df_names[['Max', 'Min']] = pd.concat([df1.astype(float).max(1).groupby(level=0).apply(list), 
#                                 df1.astype(float).min(1).groupby(level=0).apply(list)], axis=1)


# clean_data(df_names,'Max')
# df_names['doubs']=df_names['two'].apply(lambda x: re.sub(r'\[\s?\"\s?\[\s?(:"|")?','',str(x))).apply(lambda x: re.sub(r'"?\]"\]','"]',str(x))).apply(lambda x: re.sub(r'(?<=\d)"',' in"',str(x)))
# curate(df_names,'doubs','height')
# rounding_inch_feet(df_names,curation_col,'a-eghj-mo-su-z')
# # df_names
# print(len(df_names))
# match_names=df_names[['external_id',curation_col]]

In [25]:
# nexts=df[(df['blank_matches'].astype(str)=='[]')&(df['height'].astype(str)=='[]')&(df['length'].astype(str)!='[]')]
# print(len(nexts))
# nexts[['long_desc','tripple_long']]
# nexts['tripple_long']

In [26]:
# third=df[(df['na_matches'].astype(str)=='[]')&(df['height'].astype(str)=='[]')&df['split'].astype(str)=='[]']
# third_pat=r'(?i)(\d{1,3}(?:(?:\-?\s?\d+\/\d+)?(?:\.\d+)?)?.?(?!inch wide)(?:(?:Inche?s?\s?l?o?n?g?)|(?:long)))|()'
# third_pat_long=r'(?i)(\d{1,3}(?:(?:\-?\s?\d+\/\d+)?(?:\.\d+)?)?.?(?!inch wide)(?:(?:Inche?s?\s?l?o?n?g?)|(?:long)))|()'
# third['extra']=third['product_name'].apply(lambda x: re_extract(third_pat, x))
# third['long']=third['long_desc'].apply(lambda x: re_extract(third_pat_long, x))

# third_df=third[(third['extra'].astype(str)!='[]')&(third['long'].astype(str)!='[]')&(df['height'].astype(str)=='[]')&(df['length'].astype(str)=='[]')]
# third_df['extra']=third_df['extra'].apply(lambda x: str(x))
# curate(third_df, 'extra', formatted_attribute)
# print(len(third_df))

# third_na=third[(third['extra'].astype(str)=='[]')&(third['long'].astype(str)=='[]')&(third['height'].astype(str)=='[]')&(third['length'].astype(str)=='[]')]
# third_na[curation_col]='n/a'
# match_third_na=third_na[['external_id',curation_col]]
# print(len(match_third_na))

# Tripples without LWH

In [27]:
pat=r'(?i)(W x D x H)|(L x W x H)|(L\,\s?W\,\s?H)|()'
df['LWH'] = df['long_desc'].apply(lambda x: re_extract(pat, x))

In [28]:
# dfz=df[['external_id','product_name','long_desc','height','blank_matches','blank_matches_long','tripple_name','tripple_long','double_name','na_matches','LWH','trips']]

trip_height=r'''(?i)(\d+(?:\.\s?\d+)?\\{0,3}\s?(?:"|''|“|in(?:che?s?)|')?\s?x\s?\d+(?:\.\d+)?\\{0,3}\s?(?:"|''|“|in(?:che?s?)|')?\s?x\s?\d+(?:\.\s?\d+)?\\{0,3}\s?(?:"|''|“|in(?:che?s?)|')?)|()'''
df['trip_height'] = df['long_desc'].apply(lambda x: re_extract(trip_height, x))
tripple=df[(df['height'].astype(str)=='[]')&(df['LWH'].astype(str)=='[]')&(df['trips'].astype(str)=='[]')&(df['trip_height'].astype(str)!='[]')]

print(len(tripple))
tripple['trip_height_one']=tripple['trip_height'].apply(lambda x: re.sub(r'''(?<=\d)\s?"(?! \])(?!\])|('')''',' in',str(x)))
tripple['trip_height_ones']=tripple['trip_height_one'].apply(lambda x: re.sub(r'(?i)(\\{0,3}xa0)|(\\)','',str(x))).apply(lambda x: re.sub(r"(?i)((?<=\,').+x\s?)|((?<=\,\s').+x\s?)",'',str(x))).apply(lambda x: re.sub(r"(?i)((?<=\[').+x\s?)",'',str(x))).apply(lambda x: re.sub(r'''((?<=\[)')|('(?=\,))|((?<=,)\s')|('(?=\]))''','',str(x))).apply(lambda x: re.sub(r'''(?i)((?<=\[").+x\s?)''','',str(x)))
tripple['trip_height_two']=tripple['trip_height_ones'].apply(lambda x: re.sub(r"(?<=\d)\\{0,3}\s?'(?!\])(?!\,)",' ft',str(x)))
tripple=tripple[tripple['trip_height_ones'].astype(str)!='[0]']
tripple['trip_height_three']=tripple['trip_height_two'].apply(lambda x: re.sub(r'''(?i)(in(?:che?s?)?)|(\\{1,3}"(?!\]))''',' in',str(x)))
tripple['trip_height_four']=tripple['trip_height_three'].apply(lambda x: re.sub(r'','',str(x))).apply(lambda x: re.sub(r"(?<=\d)\\{0,3}'(?!\])",' ft',str(x)))
tripple['trip_height_five']=tripple['trip_height_four'].apply(lambda x: re.sub(r'''(?i)((?<=\[)(?:'|")?\d+(?:\.\d+)?\\{0,3}(?:"|''|“|\s?ft|\s?in(?:che?s?)?)?\s?x\s?\d+(?:\.\d+)?\\{0,3}(?:"|''|“|\s?ft|\s?in(?:che?s?)?)?\s?x\s?)|((?<=\,\s')\s?\d+(?:\.\d+)?\\{0,3}(?:"|''|“|\s?ft|\s?in(?:che?s?)?)?\s?x\s?\d+(?:\.\d+)?\\{0,3}(?:"|''|“|\s?ft|\s?in(?:che?s?)?)?\s?x\s?)|((?<=\,\s")\s?\d+(?:\.\d+)?\\{0,3}(?:"|''|“|\s?ft|\s?in(?:che?s?)?)?\s?x\s?\d+(?:\.\d+)?\\{0,3}(?:"|''|“|\s?ft|\s?in(?:che?s?)?)?\s?x\s?)|((?<=\,")\s?\d+(?:\.\d+)?\\{0,3}(?:"|''|“|\s?ft|\s?in(?:che?s?)?)?\s?x\s?\d+(?:\.\d+)?\\{0,3}(?:"|''|“|\s?ft|\s?in(?:che?s?)?)?\s?x\s?)|("(?=\,))|((?<=\,)")|("(?=\]))''','',str(x))).apply(lambda x: re.sub(r'((?<=\[).*\sx\s?)|(\\xa0)','',str(x)))#.apply(lambda x: re.sub(r'\\{0,3}"\s?',' in',str(x))).apply(lambda x: re.sub(r'''((?<=)\s?'\s?)|(?<!')(?<!")(?=\,)|(?<=\[)(?!')(?!")|(?<!')(?<!")(?=\])|(?:''|“)|("")''','"',str(x)))                         
tripple['trip_height_five']=tripple['trip_height_five'].apply(lambda x: re.sub(r"(?<=\[)\s?'|(?<=\,)\s?'|'\s?(?=\])|'\s?(?=\,)",'"',str(x)))

curate(tripple,'trip_height_five','height')
print(len(tripple))
rounding_inch_feet(tripple,curation_col,'a-eghj-mo-su-z')
tripple=tripple[tripple['rounding'].astype(str)=='[]']
print(len(tripple))
match_tripple=tripple[['external_id',curation_col]]

486
Repeated External IDs: []
486
470


In [29]:
print(len(df))
print(len(hmatches)+len(matchesna))#+len(match_trips)+len(match_tripple))#+len(matchesnew)+len(matchesna)+len(match_third_na)+len(third_df)+len(matchtrisplit))

39175
4341


In [30]:
dfz=df[['external_id','product_name','long_desc','height','blank_matches','blank_matches_long','tripple_name','tripple_long','double_name','na_matches','LWH','trips']]

In [31]:
pat='''(?i)((?<!\d)(?<!\.)\d+(?:\.\d+|\/\d+|(?:\s|\-)\d+\/\d+)?.{0,3}(?:''|"|inc?h?e?s?)\s?L\s?x\s?\d+(?:\.\d+|\/\d+|(?:\s|\-)\d+\/\d+)?.{0,3}(?:''|"|inc?h?e?s?)\s?W)|()'''
df['lw']=df['long_desc'].apply(lambda x: re_extract(pat, x))
pat='''(?i)(\d\\?"\s?Hx?\b)|()'''
df['H']=df['long_desc'].apply(lambda x: re_extract(pat, x))

In [32]:
h=df[(df['H'].astype(str)!='[]')]
print(len(h))

0


In [33]:
h=df[(df['H'].astype(str)!='[]')&(df['trip_height'].astype(str)!='[]')&(df['height'].astype(str)!='[]')&(df['LWH'].astype(str)!='[]')]
print(len(h))

0


In [34]:
x=df[(df['trip_height'].astype(str)=='[]')&(df['LWH'].astype(str)=='[]')&(df['height'].astype(str)=='[]')&(df['H'].astype(str)=='[]')]
pat='''(?i)((?<!\d)(?<!\.)\d+(?:\.\d+|\/\d+|(?:\s|\-)\d+\/\d+)?.{0,3}\s?.{0,3}(?:''|"|inc?h?e?s?).{0,3}\s?.{0,3}(?:tall|high|\(H\)))|((?<!\d)(?<!\.)\d+(?:\.\d+|\/\d+|(?:\s|\-)\d+\/\d+)?.{0,3}(?:''|"|inc?h?e?s?).{0,3}\s?x\s?\d+(?:\.\d+|\/\d+|(?:\s|\-)\d+\/\d+)?.{0,3}(?:''|"|inc?h?e?s?)(?!\s[LWD]\b)(?!\s\(L\sx\sW)(?!\s\(\sL\sx\sW)(?:\(?[A-z]\s?x\s?H)?)'''                               
# pat='''(\d+(?:\.\d+|\/\d+)?(?:\\\)?.{0,20})'''
x['size']=x['long_desc'].apply(lambda x: re_extract(pat, x))
height_x=x[(x['size'].astype(str)!='[]')&(x['lw'].astype(str)=='[]')]
print(len(height_x))
# size

11258


In [35]:
height_x[['long_desc','size']][0:500]
height_x[curation_col]=height_x['size'].apply(lambda x: re.sub(r'''((?<=\[').+x\s?)|(\\)''','',str(x))).apply(lambda x: re.sub(r'''(inc?h?e?s?\s?(?:tall|high|\(H\))?)|("\s?\(H\))|(")|('')''',' in',str(x))).apply(lambda x: re.sub(r'''(\s\s)''',' ',str(x))).apply(lambda x: re.sub(r'''(\s?(?:high|tall)\s?)''','',str(x))).apply(lambda x: re.sub(r'''(')''','"',str(x)))                              
height_x[['long_desc','size',curation_col]][0:500]

long_desc  \
37                                                                                                                                                                                                                                                                                                                                                                                          Safety Playpen Portable Foldable Mesh Playard Infants Toddler Fence TIME TO PLAY For little ones, it's all about fun! But keeping a constant eye on your kiddos can be hard when you got a thousand other things going on. Thankfully, Baby's portable play yard turns anywhere into a safe space to play! A FLICK OF THE WRIST Setting up your baby playpen is so easy, you might just think it's magic! We crafted our Playard for single-handed operation, so you can set it up with one hand while yore holding your kiddo with the other. NOW YOU SEE ME Peace of mind for everyone! The Baby portable playpen uses secure, see-through mesh that allows you to see baby, and baby to see you! both you and your little one will enjoy immense comfort whenever you use your indoor/outdoor play area. FUN ON THE GO Fun shouldn't have to stay at home! Whether yore in the backyard, at the park, or staying at Grandma and Grandpa, Baby portable play yards let your child enjoy playtime everywhere. And when it's time for a clean, easily remove the machine-washable lining. GREAT SIZE The Portable Playard is freestanding and portable for use inside or outdoors, measuring 55.11 inches wide and 26 inches tall and weighing only 12 lbs. DETAILS MESH SIDES & CANVAS FLOOR Featuring airy mesh sides for added visibility and a weather-resistant canvas floor to keep baby dry even on damp grass, the Portable Playard is ideal for use indoors and outdoors. COMPACT FOLD The ultra-lightweight and compact fold playard can be set up and taken down in seconds, making it perfect for use at home, a day at the park, or a weekend at the beach. TRAVEL STORAGE BAG A travel bag with a shoulder strap is included for easy use when traveling and on the go. SAFETY Positive Six-sided Design, Anti-Rollover; Anti-Collision Railing Guard to Prevent Baby colliding; when internal or external issues are detected PORTABLE This Playpen is Compact, Lightweight, and Sets Up in Seconds PERFECT FIT The Nylon Bottom Provides Protection From Rough Surfaces. The My Play 6-panel Measures AIRY MESH SIDES Easy Visibility, You Will Love that You See At a Glance How Your Baby is Doing Perfect for Use Indoor & Outdoor, Such As at Home, House Garden, in Park, A Weekend at The Beach, Travel Package includes 1 Piece Safety Playpen   
39                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [36]:
pat='''(?i)(approximate.?size.?\d+(?:\.\d+|.?\d*\/\d+)?(?:"|''|.?inc?h?e?s?)?[^\w]x(?:[^\w])\d+(?:\.\d+|.?\d*\/\d+)?(?:"|''|.?inc?h?e?s?|[^\w]))|()'''
df['size']=df['long_desc'].apply(lambda x: re_extract(pat, x))
size=df[df['size'].astype(str)!='[]']
print(len(size))
size

2


buckets external_id  \
8033  Outdoor Sculptures, Decorations, & Figurines   200647662   
8034  Outdoor Sculptures, Decorations, & Figurines   200648318   

                                                           product_name  \
8033  Designocracy Gnome Girl Bunny Easter Door Décor by J. Mills-Price   
8034   Designocracy Christmas Gnomes Holiday Door Décor by Susan Winget   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    long_desc  \
8033  G. DeBrekht Wall, Over-the-Door, and Mantelpiece Decor is made from high-quality plywood and is ready to hang. Perfect addition for a holiday home or garden decoration. High-quality sturdy & lightweight wooden decor Made from high-quality plywood UV coated for indoor and outdoor use Wall mounting hardware attached Handcrafted in the USA Some items may have slight variations due to the handcrafted nature of the art and wood grains Art by J. Mills-Price Approximate size 18 x 24 inches   
8034    G. DeBrekht Wall, Over-the-Door, and Mantelpiece Decor is made from high-quality plywood and is ready to hang. Perfect addition for a holiday home or garden decoration. High-quality sturdy & lightweight wooden decor Made from high-quality plywood UV coated for indoor and outdoor use Wall mounting hardware attached Handcrafted in the USA Some items may have slight variations due to the handcrafted nature of the art and wood grains Art by Susan Winget Approximate size 18 x 24 inches   

                                                                                                                                                                                                                                                                                                                                                                                                         LONG_DESCRIPTION  \
8033  <ul> <li>High-quality sturdy & lightweight wooden decor</li> <li>Made from high-quality plywood</li> <li>UV coated for indoor and outdoor use</li> <li>Wall mounting hardware attached</li> <li>Handcrafted in the USA</li> <li>Some items may have slight variations due to the handcrafted nature of the art and wood grains </li> <li>Art by J. Mills-Price</li> <li>Approximate size  18 x 24 inches</li> </ul>   
8034    <ul> <li>High-quality sturdy & lightweight wooden decor</li> <li>Made from high-quality plywood</li> <li>UV coated for indoor and outdoor use</li> <li>Wall mounting hardware attached</li> <li>Handcrafted in the USA</li> <li>Some items may have slight variations due to the handcrafted nature of the art and wood grains </li> <li>Art by Susan Winget</li> <li>Approximate size  18 x 24 inches</li> </ul>   

     length height width diameter blank_matches blank_matches_long  \
8033     []     []    []       []            []     [18 x 24 inch]   
8034     []     []    []       []            []     [18 x 24 inch]   

     tripple_name tripple_long double_name       double_long mm_name mm_long  \
8033           []           []          []  [18 x 24 inches]      []      []   
8034           []           []          []  [18 x 24 inches]      []      []   

     units ft_long ft_name na_matches LWH trips foot_inch trip_height  lw   H  \
8033    []      []      []         []  []    []        []          []  []  []   
8034    []      []      []         []  []    []        []          []  []  []   

                                   size  
8033  [Approximate size 18 x 24 inches]  
8034  [Approximate size 18 x 24 inches]

In [37]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(hmatches))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchesnew))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesna))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(match_third_na))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(match_third))

hmatches
matchesna


In [38]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False) 
    
looks_good('Bed Bath & Beyond', attribute,hmatches,today)

In [ ]:
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name

# def looks_good(customer, attribute,matches,today): 
#     drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False) 
    
# looks_good('Bed Bath & Beyond', attribute,match_names,today)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False) 
    
looks_good('Bed Bath & Beyond', attribute,matchesna,today)

In [ ]:
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name

# def looks_good(customer, attribute,matches,today): 
#     drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False) 
    
# looks_good('Bed Bath & Beyond', attribute,match_names_ft,today)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-match_trips.csv',index=False) 
    
looks_good('Bed Bath & Beyond', attribute,match_trips,today)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-match_tripple.csv',index=False) 
    
looks_good('Bed Bath & Beyond', attribute,match_tripple,today)